<a href="https://colab.research.google.com/github/Apolo073/Coderhouse_DataScience_EntregaFinal/blob/main/Reproducci%C3%B3n_Canciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##INTRODUCCIÓN

##DESCRIPCIÓN DEL PROBLEMA

##OBJETIVO

#### Predecir el géreno de una canción a partir las carácterísticas de la misma.

##FUENTE

Los datos utilizados en este trabajo fueron obtenidos de Kaggle y se encuentran disponibles en el siguiente enlace: https://www.kaggle.com/datasets/purumalgi/music-genre-classification?resource=download&select=train.csv

##IMPORTACIÓN DE LIBRERÍAS Y CARGA DE DATOS

In [32]:
#Se importan librerías:

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

from matplotlib.ticker import PercentFormatter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [33]:
#Se cargan Datasets:
url_train = "https://raw.githubusercontent.com/Apolo073/Coderhouse_DataScience_EntregaFinal/refs/heads/main/train.csv"
url_test = "https://raw.githubusercontent.com/Apolo073/Coderhouse_DataScience_EntregaFinal/refs/heads/main/test.csv"
url_submission = "https://raw.githubusercontent.com/Apolo073/Coderhouse_DataScience_EntregaFinal/refs/heads/main/submission.csv"

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
submission_df = pd.read_csv(url_submission)

##EXPLORACIÓN DE DATOS

### Cantidad de registros

In [34]:
train_df.shape

(17996, 17)

In [35]:
test_df.shape

(7713, 16)

In [36]:
submission_df.shape

(7713, 11)

### Tipo de Datos

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17996 entries, 0 to 17995
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         17996 non-null  object 
 1   Track Name          17996 non-null  object 
 2   Popularity          17568 non-null  float64
 3   danceability        17996 non-null  float64
 4   energy              17996 non-null  float64
 5   key                 15982 non-null  float64
 6   loudness            17996 non-null  float64
 7   mode                17996 non-null  int64  
 8   speechiness         17996 non-null  float64
 9   acousticness        17996 non-null  float64
 10  instrumentalness    13619 non-null  float64
 11  liveness            17996 non-null  float64
 12  valence             17996 non-null  float64
 13  tempo               17996 non-null  float64
 14  duration_in min/ms  17996 non-null  float64
 15  time_signature      17996 non-null  int64  
 16  Clas

In [38]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7713 entries, 0 to 7712
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Artist Name         7713 non-null   object 
 1   Track Name          7713 non-null   object 
 2   Popularity          7486 non-null   float64
 3   danceability        7713 non-null   float64
 4   energy              7713 non-null   float64
 5   key                 6905 non-null   float64
 6   loudness            7713 non-null   float64
 7   mode                7713 non-null   int64  
 8   speechiness         7713 non-null   float64
 9   acousticness        7713 non-null   float64
 10  instrumentalness    5804 non-null   float64
 11  liveness            7713 non-null   float64
 12  valence             7713 non-null   float64
 13  tempo               7713 non-null   float64
 14  duration_in min/ms  7713 non-null   float64
 15  time_signature      7713 non-null   int64  
dtypes: flo

In [39]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7713 entries, 0 to 7712
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Acoustic/Folk_0  7713 non-null   int64
 1   Alt_Music_1      7713 non-null   int64
 2   Blues_2          7713 non-null   int64
 3   Bollywood_3      7713 non-null   int64
 4   Country_4        7713 non-null   int64
 5   HipHop_5         7713 non-null   int64
 6   Indie Alt_6      7713 non-null   int64
 7   Instrumental_7   7713 non-null   int64
 8   Metal_8          7713 non-null   int64
 9   Pop_9            7713 non-null   int64
 10  Rock_10          7713 non-null   int64
dtypes: int64(11)
memory usage: 663.0 KB


Diccionario - Mapeo de género

In [40]:
# Obtener los nombres de las columnas como una lista
column_names = list(submission_df.columns)

# Crear un diccionario de mapeo usando un bucle for
mapping_dict = {}
for index, name in enumerate(column_names):
    key_part = name.split('_')[0]
    mapping_dict[index] = key_part

# Mapear la columna 'Class' a 'Genre' usando el diccionario de mapeo
train_df['Genre'] = train_df['Class'].apply(lambda x: mapping_dict.get(x))

### Primeros Registros

In [44]:
train_df.head(3)

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class,Genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.0171,NaN,0.0849,0.899,134.071,234596.0,4,5,HipHop
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.0011,0.004010,0.1010,0.569,116.454,251733.0,4,10,Rock
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.4860,0.000196,0.3940,0.787,147.681,109667.0,4,6,Indie Alt


In [42]:
test_df.head(3)

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature
0,David Bowie,Space Oddity - 2015 Remaster,73.0,0.310,0.403,NaN,-13.664,1,0.0326,0.0726,0.000093,0.1390,0.466,134.480,318027.0,4
1,Crimson Sun,Essence of Creation,34.0,0.511,0.955,1.0,-5.059,1,0.1290,0.0004,0.000009,0.2630,0.291,151.937,220413.0,4
2,P!nk,Raise Your Glass,78.0,0.700,0.709,7.0,-5.006,1,0.0839,0.0048,NaN,0.0289,0.625,122.019,202960.0,4


### Valores Nulos

### Variable objetivo

### Conclusión preliminar

## GENERALIDADES

### Hipotesis planteadas

## ANÁLISIS ESTADÍSTICO DE DATOS

### Análisis Univariado

### Análisis Bivariado

### Análisis Multivariado

## PREPROCESAMIENTO DE DATOS

### Outliers

### Valores nulos

### Eliminar columnas irrelevantes

### Encoding & Normalizado

## SELECION DE MODELOS Y ENTRENAMIENTO

## VALIDACIÓN